# Register a Method as the Original Attribute of Pandas Object

Pandas object method chaining gives us a great coding feeling without any breaking.

In many cases, it's ok via the original attributes of pandas object.
But it's possible to use our own function to handle some special cases.


- Pandas accessor register, `pandas.api.extensions.register_series_accessor` and `pandas.api.extensions.register_dataframe_accessor`
- DToolKit method register, `dtoolkit.accessor.register_series_method` and `dtoolkit.accessor.register_dataframe_method`

## Pandas Accessor Register

### Pandas Register Class

This example shows an accessor how can combine many attributes.

Just like `Series.str` accessor can access a lot of `sting` attributes, `count`, `find`, and `index`, i.e.

Copy from [pandas accessor example](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.api.extensions.register_dataframe_accessor.html).

In [ ]:
from __future__ import annotations

import pandas as pd
import numpy as np


@pd.api.extensions.register_dataframe_accessor("geo")
class GeoAccessor:
    def __init__(self, df: pd.DataFrame):
        self._obj = df

    @property
    def center(self):
        # return the geographic center point of this DataFrame
        lat = self._obj.latitude
        lon = self._obj.longitude
        return (float(lon.mean()), float(lat.mean()))

    def plot(self):
        # plot this array's data on a map, e.g., using Cartopy
        pass

In [ ]:
ds = pd.DataFrame(
    {
        "longitude": np.linspace(0, 10),
        "latitude": np.linspace(0, 20),
    }
)
ds.head()

In [ ]:
ds.geo.center

### Pandas Register Method

What if I want to register only one method?

It need to wrap class or function.

#### Wrap Class via `__call__`

In [ ]:
@pd.api.extensions.register_dataframe_accessor("col")
@pd.api.extensions.register_series_accessor("col")
class Column:
    def __init__(self, pd_obj):
        self.pd_obj = pd_obj

    def __call__(self) -> str | list[str]:
        if isinstance(self.pd_obj, pd.Series):
            return self.pd_obj.name

        return self.pd_obj.columns.tolist()

In [ ]:
ds.col()

#### Wrap function

In [ ]:
@pd.api.extensions.register_dataframe_accessor("col")
@pd.api.extensions.register_series_accessor("col")
def column(pd_obj) -> str | list[str]:
    def wrapper():
        if isinstance(pd_obj, pd.Series):
            return pd_obj.name
        return pd_obj.columns.tolist()

    return wrapper()

In [ ]:
ds.col()

### Pandas Accessor Register Conclusion

For class pandas accessor register (`pd.api.extensions.register_*_accessor`) would be great.
But for single method it would be a little bit weird.

## DToolKit Method Register

To hook single method easier.

### DToolKit Register Method

In [ ]:
from dtoolkit.accessor import register_dataframe_method
from dtoolkit.accessor import register_series_method


@register_dataframe_method("col")
@register_dataframe_method
@register_series_method("col")
@register_series_method
def column(pd_obj) -> str | list[str]:
    if isinstance(pd_obj, pd.Series):
        return pd_obj.name
    return pd_obj.columns.tolist()

Use custom accessor name `col`.

In [ ]:
ds.col()

Use accessor name `column`.

In [ ]:
ds.column()

## Extend Ability to Pandas-like Object

To extend quickly hook method as pandas-like object ability.

There are a another decorator `dtoolkit.accessor.register_method_factory`.

```python
@register_method_factory
def object_accessor(name: str | None = None):
    return pandas_like_object_accessor(name)
```

### Transform Pandas Accessor Register

In [ ]:
from dtoolkit.accessor import register_method_factory


@register_method_factory
def my_dataframe_accessor(name: str | None = None):
    return pd.api.extensions.register_dataframe_accessor(name)

In [ ]:
@my_dataframe_accessor("my_cols")
@my_dataframe_accessor
def my_columns(pd_obj: pd.DataFrame):
    return pd_obj.columns

In [ ]:
ds.my_columns()

In [ ]:
ds.my_cols()

### Transform GeoPandas Accessor Register

In [ ]:
from dtoolkit.accessor import register_method_factory
from dtoolkit.geoaccessor import register_geodataframe_accessor


@register_method_factory
def my_geodataframe_accessor(name: str | None = None):
    return register_geodataframe_accessor(name)

In [ ]:
import geopandas as gpd

from pygeos import count_coordinates, from_shapely


@my_geodataframe_accessor("my_count_coords")
@my_geodataframe_accessor
def my_count(df: gpd.GeoDataFrame):
    # Counts the number of coordinate pairs in geometry

    return df.geometry.apply(lambda x: count_coordinates(from_shapely(x)))

In [ ]:
s = gpd.GeoSeries.from_wkt(["POINT (0 0)", "POINT (1 1)", None])
s = s.to_frame("geometry")
s

In [ ]:
s.my_count()

In [ ]:
s.my_count_coords()